In [87]:
import re
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing

In [88]:
import MySQLdb
import time
conn=MySQLdb.connect("54.65.19.253", "news", "newsnews", "news")
sql = "SELECT trump.tweet, usdx FROM usdx, trump WHERE trump.timestamp = usdx.timestamp AND trump.timestamp > 1433088000;"
tStart = time.time()
data = pd.read_sql_query(sql, conn)
conn.close()
tEnd = time.time()
print("Fetch data time: " +str(tEnd-tStart)+ "sec.")

Fetch data time: 0.3688828945159912sec.


In [89]:
data = data.groupby('usdx', as_index=False).agg(lambda tweet: ' '.join(tweet))
data.head()

,usdx,tweet
0,114.0412,Watch my appearance on @Morning_Joe - great in...
1,114.3219,"""@MalachiNorris: Donald J. Trump is the very d..."
2,114.3983,"""@stevenkirk @realDonaldTrump @PatrickBuchanan..."
3,114.8460,"""@woodrowwinters @realDonaldTrump pumped for T..."
4,114.9009,"""@kolbytraveller #Trump is more than just talk..."


In [90]:
data.shape

(473, 2)

In [91]:
Y = data.as_matrix(columns=['usdx']).reshape(-1)
Y = np.gradient(Y)
#num_classes = 3
offset = np.median(Y)
Y[np.where(Y > offset)] = 1
Y[np.where(Y <= offset)] = 0
Y = Y.astype('int64')
f = np.bincount(Y)
print(f)
#print(1-((f[0]+f[1])/np.sum(f)))
#Y = shift(Y, -1, cval=1)

[237 236]


In [92]:
X = data.as_matrix(columns=['tweet'])

In [93]:
vectorizer = TfidfVectorizer(min_df=1, ngram_range=(1, 3))
X = vectorizer.fit_transform(data['tweet'].tolist())
X

<473x113689 sparse matrix of type '<class 'numpy.float64'>'
	with 184975 stored elements in Compressed Sparse Row format>

In [94]:
X= hstack([X, X.power(2)])

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
print(X_train.shape)
print(X_test.shape)

(378, 227378)
(95, 227378)


In [106]:
cv = 5
model = NuSVC(kernel='rbf',
              nu=0.5,
              degree = 5,
              verbose=0)
recall = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='recall')
precision = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='precision')
print('\nprecision:',np.mean(precision))
print('recall:', np.mean(recall))
print('acc: %.4f'%((np.bincount(np.where(cross_val_predict(model, X, Y, cv=cv, n_jobs = -1)-Y != 0, 0, 1))/len(Y))[1]))


precision: 0.542475506332
recall: 0.842553191489
acc: 0.5560


In [97]:
model.fit(X_train, y_train)
print(confusion_matrix(y_test, model.predict(X_test), labels=[0,1]))
target_names = ['down','up']
print(classification_report(y_test, model.predict(X_test), target_names=target_names))

[[29 15]
 [15 36]]
             precision    recall  f1-score   support

       down       0.66      0.66      0.66        44
         up       0.71      0.71      0.71        51

avg / total       0.68      0.68      0.68        95



In [103]:
# skf = StratifiedKFold(n_splits=5, shuffle=True)
# for train_idx, test_idx in skf.split(X, Y):
#     print ("Running Fold")
#     model = NuSVC(kernel='rbf',
#                   nu=0.5,
#                   degree = 5,
#                   verbose=0)
#     model.fit(X.toarray()[train_idx], Y[train_idx])
#     target_names = ['down','up']
#     print(classification_report(Y[test_idx], model.predict(X.toarray()[test_idx]), target_names=target_names))

Running Fold
             precision    recall  f1-score   support

       down       0.58      0.69      0.63        48
         up       0.62      0.50      0.55        48

avg / total       0.60      0.59      0.59        96

Running Fold
             precision    recall  f1-score   support

       down       0.68      0.48      0.56        48
         up       0.59      0.77      0.67        47

avg / total       0.63      0.62      0.61        95

Running Fold
             precision    recall  f1-score   support

       down       0.67      0.13      0.21        47
         up       0.52      0.94      0.67        47

avg / total       0.59      0.53      0.44        94

Running Fold
             precision    recall  f1-score   support

       down       0.50      0.06      0.11        47
         up       0.50      0.94      0.65        47

avg / total       0.50      0.50      0.38        94

Running Fold
             precision    recall  f1-score   support

       down       0.8